<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 16.4 MB/s eta 0:00:00


In [2]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [3]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-3-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [4]:
len(df)

2220299

In [5]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


# Data Preprocessing

In [6]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)

In [7]:
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [8]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

113860

In [9]:
df[df["Qty Shipped"] == 0].head(50)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
24204,4434497,10.01.23,NaN,2023FM01,44475054,EVERIT GMBH,Business Channel,,CD71519,1830 24G 12P CLASS4 POE-STOCK,.,490,Communications & Networking,Lan Switches Managed Layer 2,Networking,"0,00",0.0
35112,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E390,TC21/TC26 HC WHITE FIVE SLOT,CHARGE ONLY CRADLE FIVE DEVICES,6026,AIDC/PoS Accessories & Supplies,Docking Station,"Other (incl. AIDC/POS, V7)","0,00",0.0
35113,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E412,TC21/26 BELT CLIP HC,DISINFECTANT READY TC21/TC26 HC,5706,AIDC/PoS Barcode Scanners,Barcode Scanner Accessories,"Other (incl. AIDC/POS, V7)","0,00",0.0
40384,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58182,IPHONE 14 PRO 128GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40385,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58183,IPHONE 14 PRO 128GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40386,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58185,IPHONE 14 PRO 128GB,GOLD,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40387,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58186,IPHONE 14 PRO 256GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40388,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58187,IPHONE 14 PRO 256GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40389,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58188,IPHONE 14 PRO 256GB,GOLD,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40390,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58189,IPHONE 14 PRO 256GB,DEEP PURPLE,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0


In [10]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [11]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [12]:
len(df)

685862

In [13]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

<ipython-input-13-12c59fb326eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "purchase"] = 1


## User and sku unqiue list

In [14]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [15]:
df_1 = df[["bcn_id","sku_id","Qty Shipped", "purchase"]]

In [16]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685862 entries, 7402 to 2220294
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   bcn_id       685862 non-null  int16  
 1   sku_id       685862 non-null  int32  
 2   Qty Shipped  685862 non-null  float64
 3   purchase     685862 non-null  int64  
dtypes: float64(1), int16(1), int32(1), int64(1)
memory usage: 19.6 MB


In [17]:
df_1.head(3)

,bcn_id,sku_id,Qty Shipped,purchase
7402,4820,33457,1.0,1
7902,1144,8567,34.0,1
7903,6140,23686,3.0,1


In [18]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "Qty Shipped": "sum",
    "purchase": "sum"
}).reset_index()

In [19]:
grouped_df.sort_values(by="bcn_id",ascending=True)

,bcn_id,sku_id,Qty Shipped,purchase
0,0,2276,1.0,1
28,0,24564,1.0,1
29,0,27140,5.0,2
30,0,27167,2.0,1
31,0,27340,3.0,1
...,...,...,...,...
284193,9885,1816,6.0,2
284194,9885,28315,6.0,2
284195,9886,26057,3.0,1
284196,9887,28303,2.0,1


In [70]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()
skus

,sku_id,Sku,Product Descr1,Product Descr2,ProductGroupDescription
7402,33457,CF58186,IPHONE 14 PRO 256GB,SPACE BLACK,Smart Phones
7902,8567,406A863,27BN55U 27IN 68.58CM IPS,3840 X 2160 16 : 9 HDMI,Lcd Monitor
7903,23686,CC93753,IPAD 9TH WI-FI + CELL 64GB,10.2IN - A13 CHIP - SPACE GREY,Tablets (Slate Style)
8687,48351,Y340805,SOCKETSCAN S740 UNIVERSAL,BARCODE SCANNER BLACK,Handheld Barcode Scanner
9035,41664,I440972,CHERRY MW 4500 BLACK,WIRELESS ERGONOMIC MOUSE USB,Mouse
...,...,...,...,...,...
2219620,3735,235L458,TM-T88VI 111A0 SERIAL USB ENET,PS BLACK UK,Mobile Receipt Printer
2219668,48133,Y123756,AC ADAPTER 12V/60W,2.5X5.5MM LEVEL VI,Dc/Pos Pos Accessories
2219689,26519,CD94306,HEALTHCARE 20 SLOT BATTERY,TOASTER WHITE PSU 100-240 VAC 12,Docking Station
2220178,31069,CE90633,SBO TP T14S G2 PANDUIT GESELLSC,MC00153088 W10P,Portable/Notebook Computers


## Create sparse user x item and item x user matrices

In [20]:
sparse_item_user = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['sku_id'], grouped_df['bcn_id'])))
sparse_user_item = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['bcn_id'], grouped_df['sku_id'])))

# **ALS Model**

In [21]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## LOOCV

with K=1 so 1 item per user gets dropped out

In [22]:
x = leave_k_out_split(sparse_item_user, K=1)
sparse_item_user_train = x[0]
sparse_item_user_test = x[1]

In [23]:
y = leave_k_out_split(sparse_user_item, K=1)
sparse_user_item_train = y[0]
sparse_user_item_test = y[1]

## ALS Model param tuning

In [24]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(sparse_user_item_train)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 3.36 s, sys: 5.83 s, total: 9.19 s
Wall time: 4.83 s


ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





In [ ]:
def validate(sparse_user_item_train, sparse_user_item_test, factors=200, alpha=40, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension)
    for <<iterations>> over matrices and validate with MAP@10
    """

    model = implicit.als.AlternatingLeastSquares(factors=factors,
                                                 alpha=alpha,
                                                 iterations=iterations,
                                                 regularization=regularization)

    model.fit(sparse_user_item_train, show_progress=show_progress)

    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@10 to a library that allows repeated items in prediction
    map10 = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Confidence: {alpha:>9} - Regularization: {regularization:4.3f} - Iterations: {iterations:>2} ==> MAP@10: {map10:6.5f}")
    return map10

In [ ]:
# %%time
# best_map10 = 0
# for factors in [10, 50, 100, 150, 200]:
#   for alpha in [10, 20, 30, 40]:
#     for iterations in [10, 15, 20]:
#       for regularization in [0.01, 0.05, 0.1]:
#           map10 = validate(sparse_user_item_train, sparse_user_item_test, factors, alpha, iterations, regularization, show_progress=False)
#           if map10 > best_map10:
#             best_map10 = map10
#             best_params = {'factors': factors, 'confidence': alpha, 'iterations': iterations, 'regularization': regularization}
#             print(f"Best MAP@10 found. Updating: {best_params}")

In [25]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV


class WrappedALS(BaseEstimator):
    def __init__(self, factors=100, regularization=0.01, iterations=20, alpha=40):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.alpha = alpha
        self.als_model = AlternatingLeastSquares(
            factors=self.factors,
            regularization=self.regularization,
            iterations=self.iterations,
            calculate_training_loss=True,
        )

    def fit(self, X, y=None):
        self.als_model.fit(X)
        return self

    def predict(self, X):
        return self.als_model.user_factors.dot(self.als_model.item_factors.T)

    def score(self, X, y=None):
        map_at_k = mean_average_precision_at_k(self.als_model, X, X.T, K=10)
        return map_at_k

# Define the hyperparameter ranges
param_grid = {
    'factors': [10, 50, 100, 150, 200],
    'alpha': [40],
    'iterations': [10, 20],
    'regularization': [0.01, 0.05]
}

# Initialize WrappedALS model
wrapped_als_model = WrappedALS()

# Initialize GridSearchCV
grid_search = GridSearchCV(wrapped_als_model, param_grid, cv=3, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(sparse_user_item_train)

# Get the best ALS model and its hyperparameters
best_als_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:")
print(best_params)

print("-----------------------------------------------------------------------------------------")
print("Best model MAP@10:")
best_model = AlternatingLeastSquares(**best_params)
map_10_best = mean_average_precision_at_k(best_model, sparse_user_item_train, sparse_user_item_test, K=10)
print(map_10_best)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: ignored

In [26]:
model_1 = implicit.als.AlternatingLeastSquares(factors=100,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.09136415493089443


In [27]:
model_2 = implicit.als.AlternatingLeastSquares(factors=10,
                                                 alpha=40,
                                                 iterations=10,
                                                 regularization=0.01)
model_2.fit(sparse_user_item_train)
map10_2 = mean_average_precision_at_k(model_2,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.0416503688385526


In [28]:
model_3 = implicit.als.AlternatingLeastSquares(factors=50,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_3.fit(sparse_user_item_train)
map10_3 = mean_average_precision_at_k(model_3,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.0416503688385526


In [29]:
model_4 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_4.fit(sparse_user_item_train)
map10_4 = mean_average_precision_at_k(model_4,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.0416503688385526


In [39]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.10023792157271375


In [56]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=15,
                                                 iterations=10,
                                                 regularization=0.05)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5941 [00:00<?, ?it/s]

0.10124491360815227


## ALS Best Model Performance

In [57]:
precision_als = precision_at_k(model_1, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

  0%|          | 0/5941 [00:00<?, ?it/s]

0.20030297929641475

In [58]:
ndcg = ndcg_at_k(model_1, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

  0%|          | 0/5941 [00:00<?, ?it/s]

0.12453512314736409

## ALS Model Recommendations

In [72]:
 # Assuming you want recommendations for user with ID 10
user_id = 5000
# Get the row of the user from the sparse_user_item matrix
user_row = sparse_user_item[user_id]
# Reshape the user_row to have 1 row per user
user_items = np.reshape(user_row, (1, -1))

# Now you can call the recommend function
recommended = model_1.recommend(user_id, user_items, N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab

,sku_id,conf,Sku,Product Descr1,Product Descr2,ProductGroupDescription
0,48881.0,0.613285,Z691863,BACKUP + REPLCTN STD,RNW SUP BAS COM 12M,Back-Up Software
1,22954.0,0.604344,CC47067,TRELLIX PROTECT STANDARD,COM SUB 1Y SUB BS A:5-250,Other Security Software
2,11239.0,0.563598,5417QV3,XSP MGD WFBS SERVICE,NEW XSP 01 M - 0001 - 0025 USER,"Virus Detection, Antivirus Software"
3,48897.0,0.557640,Z691943,BACKUP ESSENT STD 2S BNDL,RNW EXP SUP BAS COM 12M,Back-Up Software
4,1765.0,0.546302,091MC22,3Y BASIC ONSITE TO 3Y PROSPT,F/OPTIPLEX 3060-3280AIO 3090U,Services And Warranties
5,11090.0,0.533977,5417JC1,WORRY FREE 9 ADVANCED,RNW LIZ 12 M - 0005 USER,"Virus Detection, Antivirus Software"
6,49025.0,0.528470,Z698689,BU FOR OFFICE365 COM,SUB UPF BILL 1Y RNW USR,Back-Up Software
7,31149.0,0.486703,CE91079,OPTIPLEX 3000 MFF TPM I3-12100T,8GB 256GB SSD W10P,Desktop/Tower Computers
8,37693.0,0.475597,CG32511,BU ESSENTIALS STD COM,ADD BAS MNT 2Y 02S,Back-Up Software
9,4187.0,0.468192,2702339,REPLACABLE BATTERY,CARTRIDGE FOR BACKUPS PRO #43,Battery For Ups


In [60]:
#Get similar items
item_id = 7
n_similar = 10
similar = model.similar_items(item_id, n_similar)
print(similar)

(array([    7, 11221,   820, 40782, 15710, 47331, 17637, 37251, 10962,
        7570], dtype=int32), array([1.        , 0.96856207, 0.96686244, 0.95412254, 0.9536304 ,
       0.9473434 , 0.94493705, 0.9437005 , 0.9383846 , 0.9379647 ],
      dtype=float32))


# BPR Model

In [73]:
from implicit.bpr import BayesianPersonalizedRanking as BPR

In [78]:
bpr_model = BPR(factors=200,
                learning_rate=0.01,
                iterations=10,
                verify_negative_samples=True)

In [79]:
bpr_model.fit(sparse_user_item_train, show_progress=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [80]:
mean_average_precision_at_k(bpr_model, sparse_user_item_train, sparse_user_item_test, K=10)

  0%|          | 0/5941 [00:00<?, ?it/s]

0.0390182695981383

## BPR Model param tuning